In [24]:
pip install tensorflow==2.10.0


  Using cached tensorflow_io_gcs_filesystem-0.31.0-cp38-cp38-win_amd64.whl (1.5 MB)
  Using cached absl_py-2.1.0-py3-none-any.whl (133 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl (26.4 MB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached termcolor-2.4.0-py3-none-any.whl (7.7 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl (26 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached grpcio-1.65.1-cp38-cp38-win_amd64.whl (4.1 MB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached Markdown-3.6-py3-none-any.whl (105 kB)
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-data-server-0.7.2:
      Successfully uninstalled tensorboard-data-server-0.7.2
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.3
    Uninstalling protobuf

In [3]:
pip install numpy==1.21.0

In [9]:
import zipfile
import os

# Define the path to the uploaded zip file and extraction directory
zip_file_path = 'C:/Users/91877/Downloads/MoodforMusic.zip'  # Update this path
extraction_dir = 'C:/path/to/extraction/directory/'  # Update this path

# Create the extraction directory if it does not exist
os.makedirs(extraction_dir, exist_ok=True)

# Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_dir)

# List the contents of the extracted directory
extracted_files = os.listdir(extraction_dir)
print(extracted_files)


['data_moods.csv', 'test', 'train']


In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import random

# Define directories
train_dir = 'C:/path/to/extraction/directory/'  # Update this path
test_dir = 'C:/path/to/extraction/directory/'    # Update this path

# Define image dimensions and batch size
img_width, img_height = 128, 128
batch_size = 32

# Create ImageDataGenerator instances for training and testing
train_datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Create generators to load images in batches
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse',
    subset='validation'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse'
)

# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Set early stopping criteria
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model using the generators
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    callbacks=[early_stopping]
)

# Plot training & validation accuracy values
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc:.2f}')

# Make predictions and generate a classification report
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = test_generator.classes

print(classification_report(y_true, y_pred_classes, target_names=list(test_generator.class_indices.keys())))

# Plot the confusion matrix
cm = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=test_generator.class_indices.keys(), yticklabels=test_generator.class_indices.keys())
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# Save the model
model.save('mood_detection_model.h5')

# Music Recommendation Engine
def recommend_music(mood):
    # Placeholder for a more sophisticated recommendation engine
    mood_to_music = {
        'happy': ['Happy by Pharrell Williams', 'Uptown Funk by Mark Ronson ft. Bruno Mars'],
        'sad': ['Someone Like You by Adele', 'Fix You by Coldplay'],
        'angry': ['Breaking the Habit by Linkin Park', 'The Way I Am by Eminem'],
        'neutral': ['Shake It Off by Taylor Swift', 'Roar by Katy Perry']
    }
    return random.choice(mood_to_music.get(mood, ['No recommendations available']))

# User Interface Development
def capture_mood_and_recommend_music(image_path):
    from tensorflow.keras.preprocessing import image
    img = image.load_img(image_path, target_size=(img_width, img_height))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0

    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction, axis=1)
    class_labels = list(test_generator.class_indices.keys())
    mood = class_labels[predicted_class[0]]

    print(f'Detected mood: {mood}')
    recommended_song = recommend_music(mood)
    print(f'Recommended song: {recommended_song}')

# Integration: capturing mood from an image and recommending music
image_path = 'path/to/sample/image.jpg'  # Update this path
capture_mood_and_recommend_music(image_path)


Found 28711 images belonging to 2 classes.
Found 7176 images belonging to 2 classes.
Found 35887 images belonging to 2 classes.
Epoch 1/10
898/898 [==============================] - 401s 446ms/step - loss: 0.5288 - accuracy: 0.7988 - val_loss: 0.5086 - val_accuracy: 0.8000
Epoch 2/10
898/898 [==============================] - 387s 431ms/step - loss: 0.5068 - accuracy: 0.8000 - val_loss: 0.5015 - val_accuracy: 0.8000
Epoch 3/10
898/898 [==============================] - 391s 435ms/step - loss: 0.5033 - accuracy: 0.8000 - val_loss: 0.5005 - val_accuracy: 0.8000
Epoch 4/10
898/898 [==============================] - 426s 475ms/step - loss: 0.4980 - accuracy: 0.7999 - val_loss: 0.5018 - val_accuracy: 0.8000
Epoch 5/10
170/898 [====>.........................] - ETA: 6:21 - loss: 0.5022 - accuracy: 0.7923

In [ ]:
# Feature Extraction and Model Training

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)


In [ ]:
#  Model Evaluation

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc:.2f}')

# Make predictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Generate a classification report
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred_classes, target_names=label_encoder.classes_))
